#### Friday, December 15, 2023

Trying this notebook in the hfpt_Dec14 container. 

Yup it still works.

A more current version of this blog post is availalbe [here](https://github.com/huggingface/blog/blob/main/llama2.md)

#### Tuesday, November 7, 2023

Ran this with the "meta-llama/Llama-2-13b-hf" model ...

#### Sunday, November 5, 2023

[Llama 2 is here - get it on Hugging Face](https://huggingface.co/blog/llama2)

The current blog post does not run here without a lot of changes ... with the below changes, it runs.

In [1]:
# docker cp /home/rob/Data3/huggingface/transformers/models--meta-llama--Llama-2-7b-hf c9b676310ea0:/home/rob/Data2/huggingface/transformers
# Successfully copied 13.5GB to c9b676310ea0:/home/rob/Data2/huggingface/transformers

# docker cp /home/rob/Data3/huggingface/transformers/models--meta-llama--Llama-2-7b-chat-hf c9b676310ea0:/home/rob/Data2/huggingface/transformers

# docker cp /home/rob/Data3/huggingface/transformers/models--meta-llama--Llama-2-13b-hf c9b676310ea0:/home/rob/Data2/huggingface/transformers
# Successfully copied 26GB to c9b676310ea0:/home/rob/Data2/huggingface/transformers

# docker cp /home/rob/Data3/huggingface/transformers/models--meta-llama--Llama-2-13b-chat-hf c9b676310ea0:/home/rob/Data2/huggingface/transformers

# !rm -rf /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-7b-hf
# rm -rf /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-13b-hf

In [1]:
# !ls /home/rob/Data2/huggingface/transformers/
!ls /root/.cache/huggingface/hub

models--bert-base-uncased  models--meta-llama--Llama-2-13b-hf  version.txt


In [2]:
from transformers import AutoTokenizer
import transformers
import torch

# model = "meta-llama/Llama-2-7b-hf"
model = "meta-llama/Llama-2-13b-hf"

tokenizer = AutoTokenizer.from_pretrained(model)


I added this next line that was not part of the blog post code ...

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# "meta-llama/Llama-2-13b-hf" => 13,948Mib VRAM
model = AutoModelForCausalLM.from_pretrained(model, load_in_8bit=True)

`AnnotionFormat` is deprecated and will be removed in v4.38. Please use `transformers.image_utils.AnnotationFormat` instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# original code from blog ...
# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model,
#     torch_dtype=torch.float16,
#     device_map="auto",
# )

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16
)


In [5]:
%%time
sequences = pipeline(
    'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)

# model = "meta-llama/Llama-2-13b-hf"
# CPU times: user 28.6 s, sys: 586 ms, total: 29.2 s
# Wall time: 29.4 s

# model = "meta-llama/Llama-2-7b-hf"
# CPU times: user 21.2 s, sys: 392 ms, total: 21.6 s
# Wall time: 21.7 s

# model = "meta-llama/Llama-2-13b-hf"
# CPU times: user 28.8 s, sys: 938 ms, total: 29.8 s
# Wall time: 30 s

CPU times: user 28.6 s, sys: 586 ms, total: 29.2 s
Wall time: 29.4 s


In [6]:
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?
I've been watching "The Sopranos" and "The Wire". I also like "The Office" and "It's Always Sunny in Philadelphia".
I'm watching "The Wire" and "The Sopranos" now. I'm also watching "The Office" and "It's Always Sunny in Philadelphia".
I'm watching "The Sopranos" and "The Wire" now. I'm also watching "The Office" and "It's Always Sunny in Philadelphia".
I'm watching "The Wire" and "The Sopranos" now. I'm also watching "The Office" and "It's Always Sunny in Philadelphia".
I'm watching "The Sopranos" and "The W
